In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import plot_tree



In [2]:
df = pd.read_csv('olympic_analysis_data.csv')
print(df.head())

  country_name country_code_2 country_code_3  year  hosting_status  \
0          USA             US            USA  2004               0   
1       Russia             RU            RUS  2004               0   
2        China             CN            CHN  2004               0   
3      Germany             DE            GER  2004               0   
4    Australia             AU            AUS  2004               0   

   population  gdp_per_capita  life_expectancy  pop_20_39_percent  \
0   293000000           54200             77.7               28.1   
1   145000000           19000             65.5               29.5   
2  1300000000            5070             72.5               34.5   
3    82100000           44200             79.3               26.8   
4    20000000           43100             80.9               28.6   

   urban_population_percent   bmi_mean  area_sq_km  democracy_score  \
0                      79.8  27.027586     9630000               10   
1                     

In [3]:
df = df.drop(['country_name', 'country_code_2', 'country_code_3', 'slug_game'], axis=1)

In [4]:
#df = df.drop(['pop_20_39_percent'], axis=1)
#df = df.drop(['urban_population_percent'], axis=1) #keep this
#df = df.drop(['bmi_mean'], axis=1)
#df = df.drop(['life_expectancy'], axis=1)

In [5]:
df_train = df[df['year'] < 2012]
df_validation = df[(df['year'] == 2012) | (df['year'] == 2016)]
df_test = df[df['year'] == 2020]
y_train = df_train['total_medal_count']
X_train = df_train.drop(['total_medal_count'], axis=1)
X_validation = df_validation.drop(['total_medal_count'], axis=1)
y_validation = df_validation['total_medal_count']
X_test = df_test.drop(['total_medal_count'], axis=1)
y_test = df_test['total_medal_count']
print(X_test.head())
#X_train = df_train.drop(['total_medal_count'], axis=1)
#print(X.head())

     year  hosting_status  population  gdp_per_capita  life_expectancy  \
906  2020               0   339000000           60200             77.2   
907  2020               0  1430000000           16300             77.7   
908  2020               0   146000000           26600             70.4   
909  2020               0    67400000           41700             79.8   
910  2020               1   126000000           40000             85.1   

     pop_20_39_percent  urban_population_percent   bmi_mean  area_sq_km  \
906               27.5                      82.7  27.027586     9830000   
907               28.6                      61.4  22.179310     9560000   
908               27.9                      74.8  25.134483    17100000   
909               26.2                      83.9  26.020690      244000   
910               20.5                      91.8  22.700000      378000   

     democracy_score  
906               10  
907               -7  
908                4  
909         

In [6]:
rf = RandomForestRegressor(n_estimators=15, random_state=0, oob_score=True)
rf.fit(X_train, y_train)




RandomForestRegressor(n_estimators=15, oob_score=True, random_state=0)

In [7]:
oob_score = rf.oob_score_
print(f'Out-of-Bag Score: {oob_score}')
y_pred = rf.predict(X_validation)

mse = mean_squared_error(y_validation,y_pred )
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_validation, y_pred)
print(f'R-squared: {r2}')


Out-of-Bag Score: 0.6960713521691784
Mean Squared Error: 76.02990379746838
R-squared: 0.821145842092631


In [8]:
'''
# Assuming regressor is your trained Random Forest model
# Pick one tree from the forest, e.g., the first tree (index 0)
tree_to_plot = rf.estimators_[0]

# Plot the decision tree
plt.figure(figsize=(20, 10))
plot_tree(tree_to_plot, feature_names=df.columns.tolist(), filled=True, rounded=True, fontsize=10)
plt.title("Decision Tree from Random Forest")
plt.show()'''

'\n# Assuming regressor is your trained Random Forest model\n# Pick one tree from the forest, e.g., the first tree (index 0)\ntree_to_plot = rf.estimators_[0]\n\n# Plot the decision tree\nplt.figure(figsize=(20, 10))\nplot_tree(tree_to_plot, feature_names=df.columns.tolist(), filled=True, rounded=True, fontsize=10)\nplt.title("Decision Tree from Random Forest")\nplt.show()'

In [9]:
y_testpred = rf.predict(X_test)


mse = mean_squared_error(y_test,y_testpred )
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test,y_testpred)
print(f'R-squared: {r2}')

Mean Squared Error: 95.72804497991967
R-squared: 0.8005555956512017


In [10]:
'''pd.DataFrame({'actual': y_test, 'predicted': y_testpred}).to_csv('predictions.csv', index=False)
print(X_test)'''

"pd.DataFrame({'actual': y_test, 'predicted': y_testpred}).to_csv('predictions.csv', index=False)\nprint(X_test)"

In [11]:
'''for est in range(10,30, 1):
    rf = RandomForestRegressor(n_estimators=est, random_state=0, oob_score=True)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_validation)
    mse = mean_squared_error(y_validation,y_pred )
    r2 = r2_score(y_validation, y_pred)
    print(f'Estimators: {est}')
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}')
    
    y_testpred = rf.predict(X_test)
    mse = mean_squared_error(y_test,y_testpred )
    r2 = r2_score(y_test,y_testpred)
    print(f'Mean Squared Error: {mse}')
    print(f'R-squared: {r2}\n')
    tree_to_plot = rf.estimators_[0]

# Plot the decision tree
    plt.figure(figsize=(20, 10))
    plot_tree(tree_to_plot, feature_names=df.columns.tolist(), filled=True, rounded=True, fontsize=10)
    plt.title("Decision Tree from Random Forest")
    plt.show()
'''

'for est in range(10,30, 1):\n    rf = RandomForestRegressor(n_estimators=est, random_state=0, oob_score=True)\n    rf.fit(X_train, y_train)\n    y_pred = rf.predict(X_validation)\n    mse = mean_squared_error(y_validation,y_pred )\n    r2 = r2_score(y_validation, y_pred)\n    print(f\'Estimators: {est}\')\n    print(f\'Mean Squared Error: {mse}\')\n    print(f\'R-squared: {r2}\')\n    \n    y_testpred = rf.predict(X_test)\n    mse = mean_squared_error(y_test,y_testpred )\n    r2 = r2_score(y_test,y_testpred)\n    print(f\'Mean Squared Error: {mse}\')\n    print(f\'R-squared: {r2}\n\')\n    tree_to_plot = rf.estimators_[0]\n\n# Plot the decision tree\n    plt.figure(figsize=(20, 10))\n    plot_tree(tree_to_plot, feature_names=df.columns.tolist(), filled=True, rounded=True, fontsize=10)\n    plt.title("Decision Tree from Random Forest")\n    plt.show()\n'